## Importing libraries

In [1]:
import pypots
import os
import sys
from pypots.utils.metrics import calc_mae
from pypots.optim import Adam
from pypots.imputation import SAITS, BRITS, USGAN, GPVAE, MRNN
import numpy as np
import benchpots
from pypots.utils.random import set_random_seed
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pandas as pd    
from pypotsModify.benchpotsMAE.datasets import preprocess_physionet2012

2025-02-12 22:04:14.905018: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-12 22:04:14.921071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-12 22:04:14.926007: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-12 22:04:14.939146: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



████████╗██╗███╗   ███╗███████╗    ███████╗███████╗██████╗ ██╗███████╗███████╗    █████╗ ██╗
╚══██╔══╝██║████╗ ████║██╔════╝    ██╔════╝██╔════╝██╔══██╗██║██╔════╝██╔════╝   ██╔══██╗██║
   ██║   ██║██╔████╔██║█████╗█████╗███████╗█████╗  ██████╔╝██║█████╗  ███████╗   ███████║██║
   ██║   ██║██║╚██╔╝██║██╔══╝╚════╝╚════██║██╔══╝  ██╔══██╗██║██╔══╝  ╚════██║   ██╔══██║██║
   ██║   ██║██║ ╚═╝ ██║███████╗    ███████║███████╗██║  ██║██║███████╗███████║██╗██║  ██║██║
   ╚═╝   ╚═╝╚═╝     ╚═╝╚══════╝    ╚══════╝╚══════╝╚═╝  ╚═╝╚═╝╚══════╝╚══════╝╚═╝╚═╝  ╚═╝╚═╝
ai4ts v0.0.3 - building AI for unified time-series analysis, https://time-series.ai 



/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)
/usr/local/lib/python3.11/dist-packages/pypots/nn/modules/reformer/local_attention.py:98: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled=False)


## Loading database

#### StandardScaler

In [2]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=1)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-12 22:04:22 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-12 22:04:22 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-12 22:04:22 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-12 22:04:22 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-12 22:04:22 [INFO]: Loaded successfully!
2025-02-12 22:04:36 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-12 22:04:36 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-12 22:04:36 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

#### MinMaxScaler

In [7]:
set_random_seed()

# Load the PhysioNet-2012 dataset
physionet2012_dataset = preprocess_physionet2012(subset="all", rate=0.1, normalization=2)

# Take a look at the generated PhysioNet-2012 dataset, you'll find that everything has been prepared for you,
# data splitting, normalization, additional artificially-missing values for evaluation, etc.
print(physionet2012_dataset.keys())

2025-02-12 22:10:11 [INFO]: Have set the random seed as 2022 for numpy and pytorch.
2025-02-12 22:10:11 [INFO]: You're using dataset physionet_2012, please cite it properly in your work. You can find its reference information at the below link: 
https://github.com/WenjieDu/TSDB/tree/main/dataset_profiles/physionet_2012
2025-02-12 22:10:11 [INFO]: Dataset physionet_2012 has already been downloaded. Processing directly...
2025-02-12 22:10:11 [INFO]: Dataset physionet_2012 has already been cached. Loading from cache directly...
2025-02-12 22:10:11 [INFO]: Loaded successfully!
2025-02-12 22:10:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-12 22:10:24 [WARNING]: Note that physionet_2012 has sparse observations in the time series, hence we don't add additional missing values to the training dataset. 
2025-02-12 22:10:24 [INFO]: 68807 values masked out in the val set as ground t

dict_keys(['n_classes', 'n_steps', 'n_features', 'scaler', 'train_X', 'train_y', 'train_ICUType', 'val_X', 'val_y', 'val_ICUType', 'test_X', 'test_y', 'test_ICUType', 'female_gender_test_X', 'female_gender_test_y', 'test_ICUType_female_gender', 'male_gender_test_X', 'male_gender_test_y', 'test_ICUType_male_gender', 'undefined_gender_test_X', 'undefined_gender_test_y', 'test_ICUType_undefined_gender', 'more_than_or_equal_to_65_test_X', 'more_than_or_equal_to_65_test_y', 'test_ICUType_more_than_or_equal_to_65', 'less_than_65_test_X', 'less_than_65_test_y', 'test_ICUType_less_than_65', 'ICUType_1_test_X', 'ICUType_1_test_y', 'test_ICUType_1', 'ICUType_2_test_X', 'ICUType_2_test_y', 'test_ICUType_2', 'ICUType_3_test_X', 'ICUType_3_test_y', 'test_ICUType_3', 'ICUType_4_test_X', 'ICUType_4_test_y', 'test_ICUType_4', 'classificacao_undefined_test_X', 'classificacao_undefined_test_y', 'test_ICUType_classificacao_undefined', 'classificacao_baixo_peso_test_X', 'classificacao_baixo_peso_test_y', 

In [8]:
# assemble the datasets for training
dataset_for_training = {
    "X": physionet2012_dataset['train_X'],
}
# assemble the datasets for validation
dataset_for_validating = {
    "X": physionet2012_dataset['val_X'],
    "X_ori": physionet2012_dataset['val_X_ori'],
}

dataset_for_testing_ori = {
    "X_ori": physionet2012_dataset['test_X_ori'],
    "female_gender_test_X_ori": physionet2012_dataset['female_gender_test_X_ori'],
    "male_gender_test_X_ori": physionet2012_dataset['male_gender_test_X_ori'],
    "undefined_gender_test_X_ori": physionet2012_dataset['undefined_gender_test_X_ori'],
    "more_than_or_equal_to_65_test_X_ori":  physionet2012_dataset['more_than_or_equal_to_65_test_X_ori'],
    "less_than_65_test_X_ori": physionet2012_dataset['less_than_65_test_X_ori'],
    "ICUType_1_test_X_ori": physionet2012_dataset['ICUType_1_test_X_ori'],
    "ICUType_2_test_X_ori": physionet2012_dataset['ICUType_2_test_X_ori'],
    "ICUType_3_test_X_ori": physionet2012_dataset['ICUType_3_test_X_ori'],
    "ICUType_4_test_X_ori": physionet2012_dataset['ICUType_4_test_X_ori'],
    "classificacao_undefined_test_X_ori": physionet2012_dataset['classificacao_undefined_test_X_ori'],
    "classificacao_baixo_peso_test_X_ori": physionet2012_dataset['classificacao_baixo_peso_test_X_ori'],
    "classificacao_normal_peso_test_X_ori": physionet2012_dataset['classificacao_normal_peso_test_X_ori'],
    "classificacao_sobrepeso_test_X_ori": physionet2012_dataset['classificacao_sobrepeso_test_X_ori'],
    "classificacao_obesidade_1_test_X_ori": physionet2012_dataset['classificacao_obesidade_1_test_X_ori'],
    "classificacao_obesidade_2_test_X_ori": physionet2012_dataset['classificacao_obesidade_2_test_X_ori'],
    "classificacao_obesidade_3_test_X_ori": physionet2012_dataset['classificacao_obesidade_3_test_X_ori']
}

# assemble the datasets for test
dataset_for_testing = {
    "X": physionet2012_dataset['test_X'],
    "female_gender_test_X": physionet2012_dataset['female_gender_test_X'],
    "male_gender_test_X": physionet2012_dataset['male_gender_test_X'],
    "undefined_gender_test_X": physionet2012_dataset['undefined_gender_test_X'],
    "more_than_or_equal_to_65_test_X":  physionet2012_dataset['more_than_or_equal_to_65_test_X'],
    "less_than_65_test_X": physionet2012_dataset['less_than_65_test_X'],
    "ICUType_1_test_X": physionet2012_dataset['ICUType_1_test_X'],
    "ICUType_2_test_X": physionet2012_dataset['ICUType_2_test_X'],
    "ICUType_3_test_X": physionet2012_dataset['ICUType_3_test_X'],
    "ICUType_4_test_X": physionet2012_dataset['ICUType_4_test_X'],
    "classificacao_undefined_test_X": physionet2012_dataset['classificacao_undefined_test_X'],
    "classificacao_baixo_peso_test_X": physionet2012_dataset['classificacao_baixo_peso_test_X'],
    "classificacao_normal_peso_test_X": physionet2012_dataset['classificacao_normal_peso_test_X'],
    "classificacao_sobrepeso_test_X": physionet2012_dataset['classificacao_sobrepeso_test_X'],
    "classificacao_obesidade_1_test_X": physionet2012_dataset['classificacao_obesidade_1_test_X'],
    "classificacao_obesidade_2_test_X": physionet2012_dataset['classificacao_obesidade_2_test_X'],
    "classificacao_obesidade_3_test_X": physionet2012_dataset['classificacao_obesidade_3_test_X']
}

## calculate the mask to indicate the ground truth positions in test_X_ori, will be used by metric funcs to evaluate models
test_X_indicating_mask = []
test_X_ori = []
for i, j in zip(dataset_for_testing_ori.values(), dataset_for_testing.values()):
    test_X_indicating_mask.append(np.isnan(i) ^ np.isnan(j))
    test_X_ori.append(np.nan_to_num(i))# metric functions do not accpet input with NaNs, hence fill NaNs with 0


In [9]:
test_X_indicating_mask_variable = []
test_X_ori_variable = []
for i in range(len(test_X_indicating_mask)):
    test_X_indicating_mask_variable.append(test_X_indicating_mask[i].reshape(37, len(test_X_indicating_mask[i]) * 48))
    test_X_ori_variable.append(test_X_ori[i].reshape(37, len(test_X_ori[i]) * 48))

## Inicialize the models

#### SAITS

In [6]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/saits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:00:52 [INFO]: No given device, using default device: cuda
2025-02-12 21:00:52 [INFO]: Model files will be saved to tutorial_results/imputation/saits/20250212_T210052
2025-02-12 21:00:52 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/saits/20250212_T210052/tensorboard
2025-02-12 21:00:52 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


In [5]:
saits = SAITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    n_layers=1,
    d_model=256,
    d_ffn=128,
    n_heads=4,
    d_k=64,
    d_v=64,
    dropout=0.1,
    ORT_weight=1,  # you can adjust the weight values of arguments ORT_weight
    # and MIT_weight to make the SAITS model focus more on one task. Usually you can just leave them to the default values, i.e. 1.
    MIT_weight=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:40 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:40 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.


2025-02-06 23:24:40 [INFO]: SAITS initialized with the given hyperparameters, the number of trainable parameters: 720,182


#### BRITS

In [16]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/brits",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:04:25 [INFO]: No given device, using default device: cuda
2025-02-12 21:04:25 [INFO]: Model files will be saved to tutorial_results/imputation/brits/20250212_T210425
2025-02-12 21:04:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/brits/20250212_T210425/tensorboard
2025-02-12 21:04:25 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


In [6]:
brits = BRITS(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:47 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:47 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:24:47 [INFO]: BRITS initialized with the given hyperparameters, the number of trainable parameters: 239,344


#### USGAN

In [29]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/us_gan",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:23:25 [INFO]: No given device, using default device: cuda
2025-02-12 21:23:25 [INFO]: Model files will be saved to tutorial_results/imputation/us_gan/20250212_T212325
2025-02-12 21:23:25 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/us_gan/20250212_T212325/tensorboard
2025-02-12 21:23:25 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


In [7]:
us_gan = USGAN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=256,
    lambda_mse=1,
    dropout=0.1,
    G_steps=1,
    D_steps=1,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    G_optimizer=Adam(lr=1e-3),
    D_optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:24:55 [INFO]: No given device, using default device: cuda
2025-02-06 23:24:55 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:24:55 [INFO]: USGAN initialized with the given hyperparameters, the number of trainable parameters: 1,258,517


#### GPVAE

In [31]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/gp_vae",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 21:42:23 [INFO]: No given device, using default device: cuda
2025-02-12 21:42:23 [INFO]: Model files will be saved to tutorial_results/imputation/gp_vae/20250212_T214223
2025-02-12 21:42:23 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/gp_vae/20250212_T214223/tensorboard
2025-02-12 21:42:23 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


In [5]:
gp_vae = GPVAE(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    latent_size=37,
    encoder_sizes=(128,128),
    decoder_sizes=(256,256),
    kernel="cauchy",
    beta=0.2,
    M=1,
    K=1,
    sigma=1.005,
    length_scale=7.0,
    kernel_scales=1,
    window_size=24,
    batch_size=32,
    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-10 22:29:48 [INFO]: No given device, using default device: cuda
2025-02-10 22:29:48 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.


2025-02-10 22:29:48 [INFO]: GPVAE initialized with the given hyperparameters, the number of trainable parameters: 229,652


#### MRNN

In [10]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    saving_path="tutorial_results/imputation/mrnn",
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-12 22:10:50 [INFO]: No given device, using default device: cuda
2025-02-12 22:10:50 [INFO]: Model files will be saved to tutorial_results/imputation/mrnn/20250212_T221050
2025-02-12 22:10:50 [INFO]: Tensorboard file will be saved to tutorial_results/imputation/mrnn/20250212_T221050/tensorboard
2025-02-12 22:10:50 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


In [9]:
mrnn = MRNN(
    n_steps=physionet2012_dataset['n_steps'],
    n_features=physionet2012_dataset['n_features'],
    rnn_hidden_size=128,

    # here we set epochs=10 for a quick demo, you can set it to 100 or more for better performance
    epochs=10,
    # here we set patience=3 to early stop the training if the evaluting loss doesn't decrease for 3 epoches.
    # You can leave it to defualt as None to disable early stopping.
    patience=3,
    # give the optimizer. Different from torch.optim.Optimizer, you don't have to specify model's parameters when
    # initializing pypots.optim.Optimizer. You can also leave it to default. It will initilize an Adam optimizer with lr=0.001.
    optimizer=Adam(lr=1e-3),
    # this num_workers argument is for torch.utils.data.Dataloader. It's the number of subprocesses to use for data loading.
    # Leaving it to default as 0 means data loading will be in the main process, i.e. there won't be subprocesses.
    # You can increase it to >1 if you think your dataloading is a bottleneck to your model training speed
    num_workers=0,
    # just leave it to default as None, PyPOTS will automatically assign the best device for you.
    # Set it as 'cpu' if you don't have CUDA devices. You can also set it to 'cuda:0' or 'cuda:1' if you have multiple CUDA devices, even parallelly on ['cuda:0', 'cuda:1']
    device=None,
    # set the path for saving tensorboard and trained model files
    # only save the best model after training finished.
    # You can also set it as "better" to save models performing better ever during training.
    model_saving_strategy="best",
)

2025-02-06 23:25:11 [INFO]: No given device, using default device: cuda
2025-02-06 23:25:11 [WARNING]: ‼️ saving_path not given. Model files and tensorboard file will not be saved.
2025-02-06 23:25:11 [INFO]: MRNN initialized with the given hyperparameters, the number of trainable parameters: 107,951


## Train/Load the models

#### SAITS

In [7]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
saits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:01:12 [INFO]: Epoch 001 - training loss: 0.1661, validation loss: 0.0078
2025-02-12 21:01:12 [INFO]: Saved the model to tutorial_results/imputation/saits/20250212_T210052/SAITS_epoch1_loss0.00782671607254694.pypots
2025-02-12 21:01:15 [INFO]: Epoch 002 - training loss: 0.1120, validation loss: 0.0064
2025-02-12 21:01:15 [INFO]: Saved the model to tutorial_results/imputation/saits/20250212_T210052/SAITS_epoch2_loss0.0063897234620526435.pypots
2025-02-12 21:01:19 [INFO]: Epoch 003 - training loss: 0.0949, validation loss: 0.0046
2025-02-12 21:01:19 [INFO]: Saved the model to tutorial_results/imputation/saits/20250212_T210052/SAITS_epoch3_loss0.004641570216820886.pypots
2025-02-12 21:01:22 [INFO]: Epoch 004 - training loss: 0.0820, validation loss: 0.0042
2025-02-12 21:01:22 [INFO]: Saved the model to tutorial_results/imputation/saits/20250212_T210052/SAITS_epoch4_loss0.004174602295582493.pypots
2025-02-12 21:01:25 [INFO]: Epoch 005 - training loss: 0.0727, validation loss: 

#### Load - Standard Scaler

In [10]:
saits.load("tutorial_results/imputation/saits/20250204_T160556/SAITS.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:325: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-0

#### Load - Min/Max Scaler

In [ ]:
saits.load("tutorial_results/imputation/saits/20250210_T210052/SAITS.pypots")

#### BRITS

In [17]:
# train the model on the training set, and validate it on the validating set to select the best model for testing in the next step
brits.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:06:06 [INFO]: Epoch 001 - training loss: 0.1748, validation loss: 0.0076
2025-02-12 21:06:06 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch1_loss0.007565443698937694.pypots
2025-02-12 21:06:57 [INFO]: Epoch 002 - training loss: 0.0984, validation loss: 0.0052
2025-02-12 21:06:57 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch2_loss0.005197163403499872.pypots
2025-02-12 21:07:49 [INFO]: Epoch 003 - training loss: 0.0821, validation loss: 0.0044
2025-02-12 21:07:49 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch3_loss0.0043778282163354255.pypots
2025-02-12 21:08:42 [INFO]: Epoch 004 - training loss: 0.0753, validation loss: 0.0038
2025-02-12 21:08:42 [INFO]: Saved the model to tutorial_results/imputation/brits/20250212_T210425/BRITS_epoch4_loss0.0038322627195157112.pypots
2025-02-12 21:09:34 [INFO]: Epoch 005 - training loss: 0.0707, validation loss

#### Load - Standard Scaler

In [11]:
brits.load("tutorial_results/imputation/brits/20250204_T162108/BRITS.pypots")

2025-02-06 23:25:24 [INFO]: Model loaded successfully from tutorial_results/imputation/brits/20250204_T162108/BRITS.pypots


#### Load - Min/Max Scaler

In [ ]:
brits.load("tutorial_results/imputation/brits/20250210_T210425/BRITS.pypots")

NameError: name 'brits' is not defined

#### USGAN

In [30]:
us_gan.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:25:15 [INFO]: Epoch 001 - generator training loss: -0.0597, discriminator training loss: 0.1872, validation loss: 0.0179
2025-02-12 21:25:15 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch1_loss0.01785113320996364.pypots
2025-02-12 21:26:37 [INFO]: Epoch 002 - generator training loss: -0.0020, discriminator training loss: 0.0545, validation loss: 0.0143
2025-02-12 21:26:37 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch2_loss0.014305492537096143.pypots
2025-02-12 21:28:01 [INFO]: Epoch 003 - generator training loss: 0.0047, discriminator training loss: 0.0385, validation loss: 0.0115
2025-02-12 21:28:01 [INFO]: Saved the model to tutorial_results/imputation/us_gan/20250212_T212325/USGAN_epoch3_loss0.011536479081648092.pypots
2025-02-12 21:29:26 [INFO]: Epoch 004 - generator training loss: 0.0015, discriminator training loss: 0.0321, validation loss: 0.0092
2025-02-12 21:29:26 [INFO]: Sav

#### Load - Standard Scaler

In [12]:
us_gan.load("tutorial_results/imputation/us_gan/20250205_T144707/USGAN.pypots")

2025-02-06 23:25:28 [INFO]: Model loaded successfully from tutorial_results/imputation/us_gan/20250205_T144707/USGAN.pypots


#### Load - Min/Max Scaler

In [ ]:
us_gan.load("tutorial_results/imputation/us_gan/20250205_T212325/USGAN.pypots")

#### GPVAE

In [32]:
gp_vae.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 21:42:40 [INFO]: Epoch 001 - training loss: 24477.7298, validation loss: 0.0211
2025-02-12 21:42:40 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch1_loss0.02106783300017317.pypots
2025-02-12 21:42:44 [INFO]: Epoch 002 - training loss: 22721.8937, validation loss: 0.0109
2025-02-12 21:42:44 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch2_loss0.010868297067160408.pypots
2025-02-12 21:42:47 [INFO]: Epoch 003 - training loss: 22712.9991, validation loss: 0.0108
2025-02-12 21:42:47 [INFO]: Saved the model to tutorial_results/imputation/gp_vae/20250212_T214223/GPVAE_epoch3_loss0.010810701572336257.pypots
2025-02-12 21:42:51 [INFO]: Epoch 004 - training loss: 22711.2009, validation loss: 0.0110
2025-02-12 21:42:55 [INFO]: Epoch 005 - training loss: 22710.5562, validation loss: 0.0109
2025-02-12 21:42:59 [INFO]: Epoch 006 - training loss: 22710.2269, validation loss: 0.0108
2025-02-12 21:42:59 [IN

#### Load - Standard Scaler

In [6]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250205_T160427/GPVAE.pypots")

/usr/local/lib/python3.11/dist-packages/pypots/base.py:324: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded_model = torch.load(path, map_location=self.device)
2025-02-1

#### Load - MinMax Scaler

In [ ]:
gp_vae.load("tutorial_results/imputation/gp_vae/20250205_T214223/GPVAE.pypots")

#### MRNN

In [11]:
mrnn.fit(train_set=dataset_for_training, val_set=dataset_for_validating)

2025-02-12 22:11:29 [INFO]: Epoch 001 - training loss: 0.1970, validation loss: 0.0284
2025-02-12 22:11:29 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250212_T221050/MRNN_epoch1_loss0.028413710246483485.pypots
2025-02-12 22:11:42 [INFO]: Epoch 002 - training loss: 0.0690, validation loss: 0.0341
2025-02-12 22:11:56 [INFO]: Epoch 003 - training loss: 0.0532, validation loss: 0.0426
2025-02-12 22:12:09 [INFO]: Epoch 004 - training loss: 0.0448, validation loss: 0.0483
2025-02-12 22:12:09 [INFO]: Exceeded the training patience. Terminating the training procedure...
2025-02-12 22:12:09 [INFO]: Finished training. The best model is from epoch#1.
2025-02-12 22:12:09 [INFO]: Saved the model to tutorial_results/imputation/mrnn/20250212_T221050/MRNN.pypots


In [14]:
mrnn.load("tutorial_results/imputation/mrnn/20250205_T165040/MRNN.pypots")

2025-02-06 23:25:34 [INFO]: Model loaded successfully from tutorial_results/imputation/mrnn/20250205_T165040/MRNN.pypots


## Testing stage

#### SAITS

In [8]:
saits_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   saits_results = saits.predict(_dict)
   saits_imputation.append(saits_results["imputation"])


In [9]:
saits_imputation_variable = []
for i in range(len(saits_imputation)):
    saits_imputation_variable.append(saits_imputation[i].reshape(37, len(saits_imputation[i]) * 48))


#### BRITS

In [18]:
brits_imputation = []
for value in dataset_for_testing.values():
    _dict = {'X':value}
    brits_results = brits.predict(_dict)
    brits_imputation.append(brits_results["imputation"])

In [19]:
brits_imputation_variable = []
for i in range(len(brits_imputation)):
    brits_imputation_variable.append(brits_imputation[i].reshape(37, len(brits_imputation[i]) * 48))

#### USGAN

In [33]:
usgan_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   usgan_results = us_gan.predict(_dict)
   usgan_imputation.append(usgan_results["imputation"])

In [34]:
usgan_imputation_variable = []
for i in range(len(usgan_imputation)):
    usgan_imputation_variable.append(usgan_imputation[i].reshape(37, len(usgan_imputation[i]) * 48))

#### GPVAE

In [35]:
gpvae_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   gpvae_results = gp_vae.predict(_dict)
   gpvae_imputation.append(gpvae_results["imputation"])

In [36]:
gpvae_imputation_variable = []
for i in range(len(gpvae_imputation)):
    gpvae_imputation_variable.append(gpvae_imputation[i].reshape(37, len(gpvae_imputation[i]) * 48))

#### MRNN

In [23]:
mrnn_imputation = []
for value in  dataset_for_testing.values():
   _dict = {'X':value}
   mrnn_results = mrnn.predict(_dict)
   mrnn_imputation.append(mrnn_results["imputation"])

In [24]:
mrnn_imputation_variable = []
for i in range(len(mrnn_imputation)):
    mrnn_imputation_variable.append(mrnn_imputation[i].reshape(37, len(mrnn_imputation[i]) * 48))

## Calculate mean absolute error

#### SAITS

In [10]:
testing_mae_saits_append_subgroups = []
testing_mae_saits_append_variables = []
for i in range(len(saits_imputation_variable)):
    for j in range(len(saits_imputation_variable[i])):
        testing_mae_saits_append_variables.append(calc_mae(saits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_saits_append_subgroups.append(testing_mae_saits_append_variables)
    testing_mae_saits_append_variables = [] 

#### BRITS

In [20]:
testing_mae_brits_append_subgroups = []
testing_mae_brits_append_variables = []
for i in range(len(brits_imputation_variable)):
    for j in range(len(brits_imputation_variable[i])):
      testing_mae_brits_append_variables.append(calc_mae(brits_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_brits_append_subgroups.append(testing_mae_brits_append_variables)
    testing_mae_brits_append_variables = []

#### USGAN

In [37]:
testing_mae_usgan_append_subgroups = []
testing_mae_usgan_append_variables = []
for i in range(len(usgan_imputation_variable)):
    for j in range(len(usgan_imputation_variable[i])):
        testing_mae_usgan_append_variables.append(calc_mae(usgan_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_usgan_append_subgroups.append(testing_mae_usgan_append_variables)
    testing_mae_usgan_append_variables = []

#### GPVAE

In [38]:
testing_mae_gpvae_append_subgroups = []
testing_mae_gpvae_append_variables = []
for i in range(len(gpvae_imputation_variable)):
    for j in range(len(gpvae_imputation_variable[i])):
        testing_mae_gpvae_append_variables.append(calc_mae(gpvae_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_gpvae_append_subgroups.append(testing_mae_gpvae_append_variables)
    testing_mae_gpvae_append_variables = []

#### MRNN

In [29]:
testing_mae_mrnn_append_subgroups = []
testing_mae_mrnn_append_variables = []
for i in range(len(mrnn_imputation_variable)):
    for j in range(len(mrnn_imputation_variable[i])):
        testing_mae_mrnn_append_variables.append(calc_mae(mrnn_imputation_variable[i][j], test_X_ori_variable[i][j], test_X_indicating_mask_variable[i][j]))
    testing_mae_mrnn_append_subgroups.append(testing_mae_mrnn_append_variables)
    testing_mae_mrnn_append_variables = []

## Results mean absolute error

In [22]:
subgroups = ["General", "Female", "Male", "Undefined Gender", "+65", "-65", "ICUType 1", "ICUType 2", "ICUType 3", "ICUType 4", "Undefined classification", "Low Weight", "Normal Weight", "Overweight", "Obesity 1", "Obesity 2", "Obesity 3"]

In [23]:
variables = ["ALP", "ALT", "AST", "Albumin", "BUN", "Bilirubin", "Cholesterol", "Creatinine", "DiasABP", "FiO2", "GCS", "Glucose", "HCO3", "HCT", "HR", "K", "Lactate", "MAP", "MechVent", "Mg", "NIDiasABP", "NIMAP", "NISysABP", "Na", "PaCO2", "PaO2", "Platelets", "RespRate", "SaO2", "SysABP", "Temp", "TroponinI", "TroponinT", "Urine", "WBC", "Weight", "Ph"]

#### SAITS

In [13]:
print("SAITS - MAE")
print("************")
for i in range(len(subgroups)): 
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_saits_append_subgroups[i][j])

SAITS - MAE
************
General
-------------
ALP : 0.031043257962657902
ALT : 0.028170038474726548
AST : 0.03196371920284396
Albumin : 0.03214313369916851
BUN : 0.029658829345883125
Bilirubin : 0.029296447671938196
Cholesterol : 0.03053975247482222
Creatinine : 0.031682666845023806
DiasABP : 0.027608199296117835
FiO2 : 0.029297089283156764
GCS : 0.03294213292290039
Glucose : 0.03134952347537376
HCO3 : 0.03186879309715356
HCT : 0.029033698823755963
HR : 0.028910497775058167
K : 0.028932415249945338
Lactate : 0.029528962856012284
MAP : 0.03083063732129137
MechVent : 0.02951229421593849
Mg : 0.033016768728603076
NIDiasABP : 0.02907365682914185
NIMAP : 0.02980853449343487
NISysABP : 0.030117783059762263
Na : 0.030114763411416532
PaCO2 : 0.028205603472151276
PaO2 : 0.0306448277088301
Platelets : 0.030168870919250548
RespRate : 0.030673072030405077
SaO2 : 0.03010541785488216
SysABP : 0.02965602938524825
Temp : 0.028850114467301468
TroponinI : 0.02876803508410699
TroponinT : 0.0277590267247

In [14]:
df_saits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_saits_mae[subgroups[i]] = testing_mae_saits_append_subgroups[i]

In [15]:
df_saits_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.031043,0.028623,0.032538,0.010592,0.027464,0.033781,0.030053,0.028138,0.030648,0.029709,0.031405,0.031685,0.028840,0.026528,0.031009,0.022108,0.035225
1,ALT,0.028170,0.030121,0.030596,0.000000,0.028983,0.031677,0.027197,0.024535,0.030832,0.030532,0.035881,0.038954,0.024859,0.025041,0.029579,0.044228,0.039819
2,AST,0.031964,0.032826,0.030105,0.041451,0.032712,0.030874,0.029192,0.027312,0.033905,0.028996,0.032522,0.034559,0.033227,0.034521,0.023238,0.026871,0.017316
3,Albumin,0.032143,0.028081,0.031695,0.007195,0.028674,0.032228,0.028690,0.026127,0.031947,0.030177,0.029346,0.033283,0.025985,0.028975,0.033340,0.022820,0.029592
4,BUN,0.029659,0.028926,0.029541,0.002651,0.032050,0.026834,0.031801,0.030840,0.030098,0.027704,0.029344,0.030355,0.027900,0.033879,0.026958,0.024370,0.034162
5,Bilirubin,0.029296,0.029505,0.031183,0.015947,0.031977,0.030812,0.032081,0.027389,0.029467,0.032614,0.031685,0.027661,0.031678,0.029220,0.020238,0.028666,0.024076
6,Cholesterol,0.030540,0.028017,0.028893,0.039825,0.029469,0.031669,0.031251,0.029139,0.034777,0.030328,0.030695,0.024737,0.026716,0.025641,0.030717,0.027538,0.035272
7,Creatinine,0.031683,0.030372,0.031353,0.023020,0.032044,0.030075,0.030371,0.027093,0.032746,0.029945,0.032148,0.020514,0.023384,0.032193,0.026546,0.031983,0.028022
8,DiasABP,0.027608,0.031772,0.030191,0.086277,0.030332,0.032991,0.029302,0.024977,0.030985,0.030234,0.031524,0.029796,0.026987,0.028361,0.026734,0.026972,0.020253
9,FiO2,0.029297,0.030811,0.029923,0.035544,0.030468,0.031878,0.029896,0.024034,0.032937,0.031624,0.028620,0.025043,0.026148,0.025213,0.031985,0.033722,0.023184


Minimum MAE value in each subgroup

In [35]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
ALT
0.22043134772111558
--------------------
Female
NIDiasABP
0.21625072772560813
--------------------
Male
PaCO2
0.2108230852051344
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
PaO2
0.2052877660438494
--------------------
-65
RespRate
0.2195340976855963
--------------------
ICUType 1
Creatinine
0.21415215930457915
--------------------
ICUType 2
Ph
0.19352872186986245
--------------------
ICUType 3
PaCO2
0.2245388907288932
--------------------
ICUType 4
HCT
0.22102052107602238
--------------------
Undefined classification
TroponinT
0.22312300711184532
--------------------
Low Weight
K
0.1351692709542507
--------------------
Normal Weight
Cholesterol
0.1852442869645915
--------------------
Overweight
MAP
0.18507979199527627
--------------------
Obesity 1
Platelets
0.16087222354644645
--------------------
Obesity 2
WBC
0.1394584956765725
--------------------
Obesity 3
AST
0.17815062101412324
--------------------


Maximum MAE value in each subgroup

In [37]:
for i in range(len(subgroups)):
    value = df_saits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_saits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")    

General
Albumin
0.30293279432474296
--------------------
Female
Urine
0.30370237839850694
--------------------
Male
Albumin
0.308384918688299
--------------------
Undefined Gender
HCT
0.5138421868725681
--------------------
+65
BUN
0.2778384531342736
--------------------
-65
DiasABP
0.3696399065245959
--------------------
ICUType 1
PaO2
0.3504441683775851
--------------------
ICUType 2
NIMAP
0.2556318391589281
--------------------
ICUType 3
PaO2
0.30687231283103145
--------------------
ICUType 4
MAP
0.3212451658407674
--------------------
Undefined classification
HCO3
0.29353148208906726
--------------------
Low Weight
NIMAP
0.6064582243175376
--------------------
Normal Weight
Glucose
0.2738133891557834
--------------------
Overweight
WBC
0.27807657394047125
--------------------
Obesity 1
HCO3
0.8932678493057511
--------------------
Obesity 2
SaO2
0.4265312900933624
--------------------
Obesity 3
NIMAP
0.4806281971702897
--------------------


#### BRITS

In [24]:
print("BRITS - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_brits_append_subgroups[i][j])

BRITS - MAE
************
General
-------------
ALP : 0.02714271451614744
ALT : 0.023480500236732458
AST : 0.02872805607407688
Albumin : 0.02660760645174079
BUN : 0.025555022701653395
Bilirubin : 0.025085490378176457
Cholesterol : 0.026067523792760033
Creatinine : 0.02628105625008944
DiasABP : 0.023210803510269006
FiO2 : 0.024787476533681622
GCS : 0.027735153166096392
Glucose : 0.025574421054083116
HCO3 : 0.02590288987025958
HCT : 0.025094551013020577
HR : 0.02439231563245925
K : 0.02443751426771262
Lactate : 0.02509530837204354
MAP : 0.025308193208257664
MechVent : 0.025884273229850978
Mg : 0.027852336603015535
NIDiasABP : 0.02431355283328725
NIMAP : 0.025348164389083157
NISysABP : 0.026094541170267913
Na : 0.024590653588609283
PaCO2 : 0.02344621102572964
PaO2 : 0.025793912716149007
Platelets : 0.025184702786608178
RespRate : 0.025502672447018395
SaO2 : 0.0266654004802727
SysABP : 0.025151581834592308
Temp : 0.025333080413934686
TroponinI : 0.02481646924598073
TroponinT : 0.02324161364

In [25]:
df_brits_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_brits_mae[subgroups[i]] = testing_mae_brits_append_subgroups[i]

In [26]:
df_brits_mae 

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.027143,0.023865,0.028980,0.026158,0.022670,0.029339,0.026517,0.022353,0.027733,0.025957,0.025533,0.024813,0.024857,0.023757,0.022323,0.024571,0.025567
1,ALT,0.023481,0.024359,0.024972,0.000000,0.025002,0.025626,0.023376,0.022184,0.026271,0.025537,0.029118,0.038295,0.022817,0.021699,0.027252,0.040437,0.025803
2,AST,0.028728,0.029300,0.024494,0.047039,0.027150,0.026433,0.025204,0.023159,0.028553,0.026468,0.026794,0.036916,0.024727,0.029179,0.025607,0.029498,0.016279
3,Albumin,0.026608,0.024297,0.027787,0.026570,0.023543,0.026717,0.022517,0.019266,0.027394,0.025498,0.023696,0.026375,0.024139,0.024101,0.027507,0.022169,0.019977
4,BUN,0.025555,0.023312,0.025848,0.005104,0.027621,0.023463,0.027244,0.024141,0.026287,0.025105,0.026822,0.031382,0.026787,0.029608,0.022936,0.016818,0.032019
5,Bilirubin,0.025085,0.026172,0.027940,0.008972,0.027005,0.027341,0.027698,0.023248,0.025307,0.029770,0.026172,0.027208,0.025900,0.023641,0.021934,0.024099,0.019053
6,Cholesterol,0.026068,0.023403,0.025137,0.004331,0.024327,0.026794,0.024596,0.023912,0.031420,0.025634,0.026880,0.013572,0.020219,0.021707,0.024317,0.025615,0.032728
7,Creatinine,0.026281,0.027577,0.025914,0.018251,0.027448,0.026724,0.024725,0.022676,0.027988,0.025826,0.025810,0.023072,0.021942,0.025095,0.019598,0.028760,0.023091
8,DiasABP,0.023211,0.027438,0.025777,0.041786,0.026233,0.028846,0.024646,0.021027,0.027520,0.027139,0.028463,0.033506,0.024738,0.023621,0.022482,0.023681,0.016888
9,FiO2,0.024787,0.026190,0.026058,0.024295,0.026053,0.026787,0.025776,0.020237,0.027264,0.028126,0.024598,0.024788,0.024445,0.021385,0.029220,0.029489,0.023752


Minimum MAE value in each subgroup

In [27]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
DiasABP
0.023210803510269006
--------------------
Female
HR
0.02170930024585297
--------------------
Male
PaCO2
0.02121252137693231
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
PaO2
0.021873438571089903
--------------------
-65
RespRate
0.022557213384951546
--------------------
ICUType 1
Platelets
0.02121699374131386
--------------------
ICUType 2
Albumin
0.019265743044393994
--------------------
ICUType 3
PaCO2
0.02181391471455736
--------------------
ICUType 4
Mg
0.022310742978122862
--------------------
Undefined classification
Albumin
0.023695818782378485
--------------------
Low Weight
Cholesterol
0.013572236206876254
--------------------
Normal Weight
K
0.019791636654028975
--------------------
Overweight
PaO2
0.01872537923021046
--------------------
Obesity 1
Glucose
0.01850029063408773
--------------------
Obesity 2
Ph
0.01496010947939735
--------------------
Obesity 3
MAP
0.014821268829400406
--------------------


Maximum MAE value in each subgroup

In [28]:
for i in range(len(subgroups)):
    value = df_brits_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_brits_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
AST
0.02872805607407688
--------------------
Female
Urine
0.030826593914607115
--------------------
Male
ALP
0.028979992498799546
--------------------
Undefined Gender
HCO3
0.055866590317831914
--------------------
+65
WBC
0.028634347212477502
--------------------
-65
ALP
0.029339381519476116
--------------------
ICUType 1
PaO2
0.039411086067656434
--------------------
ICUType 2
Lactate
0.028642889016280933
--------------------
ICUType 3
Glucose
0.032611812596924805
--------------------
ICUType 4
NIMAP
0.031616037845466814
--------------------
Undefined classification
GCS
0.029762598824543622
--------------------
Low Weight
NIDiasABP
0.05579793822957305
--------------------
Normal Weight
Glucose
0.031969489846970914
--------------------
Overweight
BUN
0.029607539442910648
--------------------
Obesity 1
TroponinT
0.02922995482977453
--------------------
Obesity 2
Weight
0.04383374057991563
--------------------
Obesity 3
GCS
0.04242287760213836
--------------------


#### USGAN 

In [39]:
print("USGAN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_usgan_append_subgroups[i][j])

USGAN - MAE
************
General
-------------
ALP : 0.048090760598627214
ALT : 0.047518540747752844
AST : 0.0528141654728246
Albumin : 0.050261664541246885
BUN : 0.0479493037969636
Bilirubin : 0.0478109439022277
Cholesterol : 0.05000755138992897
Creatinine : 0.05112939095418578
DiasABP : 0.046178559479375464
FiO2 : 0.04640046021830351
GCS : 0.05001800588481442
Glucose : 0.05180037764554821
HCO3 : 0.05217928120828053
HCT : 0.050375681230195146
HR : 0.04826684769599868
K : 0.04473846666761958
Lactate : 0.05000427789918519
MAP : 0.04854829188920495
MechVent : 0.05244303534828315
Mg : 0.05296064324105656
NIDiasABP : 0.04738450810729243
NIMAP : 0.04954867928636601
NISysABP : 0.050309677431392565
Na : 0.04973596459056172
PaCO2 : 0.045824005586180896
PaO2 : 0.04941172737318465
Platelets : 0.047924054127276815
RespRate : 0.05039880987016525
SaO2 : 0.050683959393723575
SysABP : 0.04957280942746781
Temp : 0.04608567922099825
TroponinI : 0.04949843710482529
TroponinT : 0.04701579673009741
Urine 

In [40]:
df_usgan_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_usgan_mae[subgroups[i]] = testing_mae_usgan_append_subgroups[i]

In [41]:
df_usgan_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.048091,0.047769,0.049732,0.069907,0.045890,0.050360,0.046919,0.042237,0.050245,0.049199,0.051793,0.040089,0.045702,0.045469,0.047182,0.041322,0.041180
1,ALT,0.047519,0.052544,0.049266,0.000000,0.049624,0.051444,0.055548,0.039688,0.060174,0.049737,0.053766,0.054279,0.044201,0.046264,0.047051,0.065129,0.040968
2,AST,0.052814,0.054886,0.047059,0.060711,0.053063,0.052826,0.050296,0.040769,0.052789,0.046085,0.051192,0.070347,0.049577,0.049506,0.051947,0.047563,0.040414
3,Albumin,0.050262,0.049665,0.051773,0.042330,0.043898,0.050810,0.049131,0.040182,0.049738,0.048331,0.049783,0.053351,0.041620,0.046054,0.050861,0.050700,0.051937
4,BUN,0.047949,0.047674,0.044950,0.099378,0.049237,0.048940,0.047210,0.044901,0.046677,0.047250,0.055089,0.053554,0.045494,0.054564,0.047246,0.041745,0.049600
5,Bilirubin,0.047811,0.048862,0.055236,0.046848,0.051220,0.049989,0.072274,0.041582,0.050550,0.045559,0.050764,0.043202,0.051989,0.056625,0.042198,0.046241,0.041801
6,Cholesterol,0.050008,0.044044,0.047279,0.037059,0.049317,0.049211,0.048919,0.046658,0.060583,0.047153,0.053398,0.033230,0.042449,0.051382,0.040569,0.038370,0.070656
7,Creatinine,0.051129,0.052925,0.049480,0.039101,0.054209,0.051652,0.052387,0.044138,0.052788,0.047151,0.057826,0.052388,0.044422,0.044420,0.040197,0.041925,0.045026
8,DiasABP,0.046179,0.052791,0.048887,0.126933,0.048032,0.051426,0.050826,0.043076,0.051096,0.051675,0.051990,0.052068,0.047778,0.052702,0.050864,0.056079,0.033494
9,FiO2,0.046400,0.048131,0.048779,0.039222,0.047685,0.050657,0.053697,0.040993,0.055344,0.049982,0.047977,0.042583,0.049079,0.044583,0.053420,0.052662,0.036464


Minimum MAE value in each subgroup

In [42]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
K
0.04473846666761958
--------------------
Female
Cholesterol
0.04404381253657575
--------------------
Male
PaCO2
0.04474763043551649
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Weight
0.04129227431228674
--------------------
-65
RespRate
0.04368375209028047
--------------------
ICUType 1
NISysABP
0.04164438888543889
--------------------
ICUType 2
MechVent
0.037201126807215386
--------------------
ICUType 3
TroponinT
0.045636755268445295
--------------------
ICUType 4
WBC
0.04277165317246901
--------------------
Undefined classification
TroponinT
0.04628365696999337
--------------------
Low Weight
K
0.02937946787016956
--------------------
Normal Weight
NIDiasABP
0.038985869519247
--------------------
Overweight
PaCO2
0.03894113772894867
--------------------
Obesity 1
Ph
0.03414219300908816
--------------------
Obesity 2
Ph
0.03357290784723323
--------------------
Obesity 3
DiasABP
0.03349438929653913
--------------------


Maximum MAE value in each subgroup

In [43]:
for i in range(len(subgroups)):
    value = df_usgan_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_usgan_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Mg
0.05296064324105656
--------------------
Female
MAP
0.05677509378339917
--------------------
Male
Bilirubin
0.05523572397582898
--------------------
Undefined Gender
DiasABP
0.12693270047493016
--------------------
+65
MechVent
0.057695967869961166
--------------------
-65
Glucose
0.056918280828891485
--------------------
ICUType 1
SysABP
0.09464095167411918
--------------------
ICUType 2
GCS
0.05075960704270495
--------------------
ICUType 3
HR
0.06392371835726103
--------------------
ICUType 4
SysABP
0.059539992399730506
--------------------
Undefined classification
MechVent
0.05941179715652995
--------------------
Low Weight
TroponinI
0.09469198543590841
--------------------
Normal Weight
Glucose
0.057020469869136926
--------------------
Overweight
Bilirubin
0.05662465584077266
--------------------
Obesity 1
Na
0.06700283247645569
--------------------
Obesity 2
TroponinI
0.08278429216698521
--------------------
Obesity 3
Cholesterol
0.07065558898885901
-------------------

#### GPVAE

In [44]:
print("GPVAE - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_gpvae_append_subgroups[i][j])

GPVAE - MAE
************
General
-------------
ALP : 0.06389728089205512
ALT : 0.06200686364943095
AST : 0.06931117774731774
Albumin : 0.06645271910367218
BUN : 0.061597386446482175
Bilirubin : 0.06097308521505829
Cholesterol : 0.06137191467494687
Creatinine : 0.06256293326570771
DiasABP : 0.06338489311394617
FiO2 : 0.06204592347296474
GCS : 0.06545908487583825
Glucose : 0.061059937931093065
HCO3 : 0.06136675072754152
HCT : 0.06077179967049385
HR : 0.060905584333217064
K : 0.06269134186485557
Lactate : 0.061450844058573185
MAP : 0.06571256769177909
MechVent : 0.06407099677931423
Mg : 0.060253784199208696
NIDiasABP : 0.06031539029380784
NIMAP : 0.06330946357076687
NISysABP : 0.06595177171703154
Na : 0.06409687720916689
PaCO2 : 0.056608179708824445
PaO2 : 0.06255209545415645
Platelets : 0.060653852442719106
RespRate : 0.0626147827251596
SaO2 : 0.06330294664287239
SysABP : 0.06398454116195525
Temp : 0.0632957605887654
TroponinI : 0.06502647567068906
TroponinT : 0.06487500328315915
Urine :

In [45]:
df_gpvae_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_gpvae_mae[subgroups[i]] = testing_mae_gpvae_append_subgroups[i]

In [46]:
df_gpvae_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.063897,0.059437,0.065928,0.032804,0.056694,0.062598,0.056154,0.058312,0.063287,0.061440,0.062428,0.069563,0.065745,0.057847,0.056957,0.056005,0.066125
1,ALT,0.062007,0.063044,0.063158,0.000000,0.067636,0.063150,0.068381,0.051773,0.068130,0.071302,0.069409,0.067473,0.061727,0.059457,0.059097,0.077887,0.074788
2,AST,0.069311,0.068104,0.065112,0.010881,0.066927,0.065014,0.058216,0.055112,0.069387,0.063949,0.065231,0.078250,0.062935,0.066156,0.056257,0.058342,0.060907
3,Albumin,0.066453,0.063221,0.068290,0.058597,0.062947,0.064884,0.055656,0.049724,0.065328,0.071496,0.064892,0.063617,0.057201,0.059391,0.061136,0.064766,0.050372
4,BUN,0.061597,0.063415,0.059580,0.039983,0.069014,0.061563,0.062677,0.061503,0.064571,0.061572,0.061620,0.074410,0.070304,0.067884,0.056655,0.050449,0.068208
5,Bilirubin,0.060973,0.063917,0.064020,0.110394,0.063593,0.063047,0.065783,0.059044,0.065754,0.069108,0.062679,0.050228,0.060449,0.059965,0.045802,0.061781,0.051788
6,Cholesterol,0.061372,0.061575,0.061728,0.018331,0.060900,0.062294,0.065332,0.056367,0.073159,0.057704,0.063020,0.047167,0.055717,0.052945,0.051754,0.067114,0.076350
7,Creatinine,0.062563,0.061329,0.069655,0.037039,0.061630,0.068253,0.062374,0.055551,0.065418,0.069943,0.065956,0.070789,0.060194,0.054567,0.053908,0.052027,0.081267
8,DiasABP,0.063385,0.064488,0.061734,0.210331,0.062583,0.072674,0.063540,0.052814,0.068537,0.066590,0.071308,0.071438,0.061773,0.056843,0.056950,0.062889,0.057102
9,FiO2,0.062046,0.071833,0.059928,0.058587,0.061136,0.063853,0.063909,0.055042,0.068831,0.071247,0.072242,0.066377,0.058779,0.060282,0.064618,0.070899,0.046362


Minimum MAE value in each subgroup

In [47]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
PaCO2
0.056608179708824445
--------------------
Female
ALP
0.05943705556438818
--------------------
Male
PaCO2
0.05667613500395072
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
0.05565149489069709
--------------------
-65
Mg
0.059004120155529874
--------------------
ICUType 1
Platelets
0.05510620406104771
--------------------
ICUType 2
Mg
0.0488913377924879
--------------------
ICUType 3
PaCO2
0.05234645843986417
--------------------
ICUType 4
Cholesterol
0.05770430003196169
--------------------
Undefined classification
NIDiasABP
0.05919388716671923
--------------------
Low Weight
Platelets
0.041563009396780023
--------------------
Normal Weight
Mg
0.05085602784017245
--------------------
Overweight
PaCO2
0.047466605986953805
--------------------
Obesity 1
Bilirubin
0.04580181353821788
--------------------
Obesity 2
WBC
0.04208152342192024
--------------------
Obesity 3
FiO2
0.046362065784707064
--------------------


Maximum MAE value in each subgroup

In [48]:
for i in range(len(subgroups)):
    value = df_gpvae_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_gpvae_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
AST
0.06931117774731774
--------------------
Female
FiO2
0.071832550791953
--------------------
Male
Creatinine
0.06965518953887552
--------------------
Undefined Gender
DiasABP
0.21033088366169542
--------------------
+65
BUN
0.06901402131591192
--------------------
-65
TroponinI
0.07859482135681735
--------------------
ICUType 1
PaO2
0.08124137487348888
--------------------
ICUType 2
Lactate
0.06164341253363296
--------------------
ICUType 3
TroponinI
0.07998017140527885
--------------------
ICUType 4
TroponinI
0.07497044203952626
--------------------
Undefined classification
GCS
0.07334575895254153
--------------------
Low Weight
TroponinT
0.10842176540684564
--------------------
Normal Weight
PaO2
0.07129215410887764
--------------------
Overweight
Mg
0.06900237369132736
--------------------
Obesity 1
PaCO2
0.07076721832273784
--------------------
Obesity 2
Na
0.10751933736913757
--------------------
Obesity 3
GCS
0.08768001772404654
--------------------


#### MRNN

In [53]:
print("MRNN - MAE")
print("************")
for i in range(len(subgroups)):
    print(subgroups[i]) 
    print("-------------")
    for j in range(len(variables)):
        print(variables[j], ":" ,testing_mae_mrnn_append_subgroups[i][j])

MRNN - MAE
************
General
-------------
ALP : 0.6891042237096028
ALT : 0.6697312359094596
AST : 0.7416927518816129
Albumin : 0.7463954870157028
BUN : 0.6355481879591666
Bilirubin : 0.6416969716355293
Cholesterol : 0.6727495200279108
Creatinine : 0.6716671861362805
DiasABP : 0.6808283782616357
FiO2 : 0.6227840389728461
GCS : 0.6692272659201319
Glucose : 0.6720891461117408
HCO3 : 0.6589864147836295
HCT : 0.6458595741032642
HR : 0.6621585497680288
K : 0.6896896911599226
Lactate : 0.6430019270508706
MAP : 0.6848244816182206
MechVent : 0.6811896916910888
Mg : 0.650124389963784
NIDiasABP : 0.6215348600145024
NIMAP : 0.6550581111501786
NISysABP : 0.6952255969268104
Na : 0.6844226542870258
PaCO2 : 0.6287560867465835
PaO2 : 0.6655651204171794
Platelets : 0.6457055708189681
RespRate : 0.6806955443585905
SaO2 : 0.6720348938943402
SysABP : 0.7056722589941172
Temp : 0.680304816992715
TroponinI : 0.6713200044286652
TroponinT : 0.6750343726015907
Urine : 0.7106147217278526
WBC : 0.6661850898761

In [54]:
df_mrnn_mae = pd.DataFrame(variables)

for i in range(len(subgroups)):
    df_mrnn_mae[subgroups[i]] = testing_mae_mrnn_append_subgroups[i]

In [55]:
df_mrnn_mae

,0,General,Female,Male,Undefined Gender,+65,-65,ICUType 1,ICUType 2,ICUType 3,ICUType 4,Undefined classification,Low Weight,Normal Weight,Overweight,Obesity 1,Obesity 2,Obesity 3
0,ALP,0.689104,0.667335,0.696357,0.507509,0.646565,0.687002,0.672778,0.620173,0.665549,0.670807,0.726986,0.701119,0.628677,0.624297,0.645220,0.575216,0.737773
1,ALT,0.669731,0.653103,0.654165,0.000000,0.725275,0.686937,0.749737,0.582993,0.685512,0.722545,0.730973,0.722608,0.655463,0.628625,0.628376,0.931739,0.845520
2,AST,0.741693,0.756573,0.673059,0.083308,0.705751,0.681755,0.695791,0.634816,0.798241,0.634615,0.712305,0.859331,0.675043,0.669676,0.591304,0.582306,0.681127
3,Albumin,0.746395,0.704606,0.727019,1.014342,0.673047,0.653678,0.674022,0.541006,0.717779,0.665241,0.684194,0.676810,0.665176,0.608718,0.683893,0.600455,0.590899
4,BUN,0.635548,0.699937,0.587745,0.475151,0.725550,0.627522,0.714661,0.647397,0.701698,0.619873,0.671836,0.772429,0.699477,0.665936,0.619232,0.630307,0.729258
5,Bilirubin,0.641697,0.700051,0.677235,0.408022,0.676139,0.655981,0.633770,0.604834,0.775192,0.679092,0.706135,0.533569,0.639532,0.638812,0.480346,0.669950,0.587823
6,Cholesterol,0.672750,0.669457,0.653943,0.269789,0.681913,0.652739,0.737699,0.605656,0.750578,0.646315,0.678274,0.646438,0.625275,0.602681,0.651379,0.839308,0.649382
7,Creatinine,0.671667,0.656672,0.728552,0.425493,0.649219,0.734455,0.650856,0.618545,0.738558,0.676408,0.698230,0.642483,0.664061,0.599628,0.655849,0.603157,0.854659
8,DiasABP,0.680828,0.680967,0.678367,0.561185,0.643803,0.815644,0.633256,0.588678,0.741413,0.635030,0.744099,0.770286,0.596444,0.554561,0.598039,0.468811,0.654385
9,FiO2,0.622784,0.697946,0.599732,0.278597,0.640812,0.669459,0.712953,0.560267,0.748908,0.682014,0.674755,0.651656,0.583494,0.645466,0.603720,0.625158,0.472036


Minimum MAE value in each subgroup

In [56]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].min()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
NIDiasABP
0.6215348600145024
--------------------
Female
TroponinT
0.62921742531125
--------------------
Male
PaCO2
0.5871804208287293
--------------------
Undefined Gender
Ph
0.0
--------------------
+65
Na
0.5893284527545951
--------------------
-65
NIDiasABP
0.6003298402934673
--------------------
ICUType 1
Temp
0.6057869251152296
--------------------
ICUType 2
MAP
0.5319993160382074
--------------------
ICUType 3
PaCO2
0.6184535399366906
--------------------
ICUType 4
NIDiasABP
0.593098916398076
--------------------
Undefined classification
HCT
0.6469828467080323
--------------------
Low Weight
Urine
0.4873191634699364
--------------------
Normal Weight
Mg
0.5709812876480205
--------------------
Overweight
MAP
0.526813176022993
--------------------
Obesity 1
Bilirubin
0.48034552174968403
--------------------
Obesity 2
WBC
0.446878399316957
--------------------
Obesity 3
FiO2
0.47203633593304234
--------------------


Maximum MAE value in each subgroup

In [57]:
for i in range(len(subgroups)):
    value = df_mrnn_mae[subgroups[i]].max()
    print(subgroups[i])
    for j in range(len(variables)):
        if(df_mrnn_mae[subgroups[i]][j] == value):
            var = variables[j]
    print(var)
    print(value)        
    print("--------------------")
        

General
Albumin
0.7463954870157028
--------------------
Female
AST
0.7565732521313827
--------------------
Male
Creatinine
0.7285521064107695
--------------------
Undefined Gender
MechVent
1.0643811764346358
--------------------
+65
BUN
0.7255498613054194
--------------------
-65
DiasABP
0.8156441578600511
--------------------
ICUType 1
HCT
0.8218244252064445
--------------------
ICUType 2
K
0.6633662789027489
--------------------
ICUType 3
TroponinI
0.842596466376899
--------------------
ICUType 4
SysABP
0.7555944034928384
--------------------
Undefined classification
SysABP
0.7544810939173036
--------------------
Low Weight
TroponinI
1.0675889203929847
--------------------
Normal Weight
Temp
0.7669488691837123
--------------------
Overweight
RespRate
0.7106134022293207
--------------------
Obesity 1
HCO3
1.2535198925354494
--------------------
Obesity 2
Na
1.0899931865042596
--------------------
Obesity 3
RespRate
0.961257497137432
--------------------
